# DEEPX Tutorial 05 - DX-Compiler Workflow

In Tutorial 5, you will practice compiling classification, object detection, and segmentation models using the DX-Compiler. We will also review the guide for troubleshooting problems during compilation.

The overall compilation process consists of the following steps:
1. Obtain a pre-trained model using the PyTorch framework
2. Convert the model to ONNX format
3. Compile the ONNX model into DXNN format

<img src="assets/dx-com-workflow.jpg" style="max-width: 800px;">

For more details, refer to the DX-Compiler user guide 👉 [Download](https://developer.deepx.ai/?files=MjY0NA==)

> Note: To download the User Guide, you must log in to https://developer.deepx.ai/ first.

> This tutorial is based on dx-all-suite v2.1.0, released in December 2025.

## 1. Compiling Image Classification Model (MobileNetV2)

In this section, we will walk through the complete compilation workflow
for an image classification model using MobileNetV2.

You will learn how to:
- Export a PyTorch model to ONNX
- Inspect model inputs using Netron or DXTron
- Configure preprocessing and calibration settings using JSON
- Compile the model into DXNN format using DX-Compiler

Overall process is:
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

### 1.1. Export PyTorch → ONNX

In [ ]:
# Install pytorch & onnx
!pip install --quiet torch torchvision onnx onnxsim onnxscript portpicker tqdm seaborn

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

#### 1.1.1. Export PyTorch based MobileNetV2 model to ONNX
 - Expected result: The file `MobilenetV2.onnx` will be generated

In [ ]:
import torch, torchvision

# Load MobileNetV2 model
model = torchvision.models.mobilenet_v2(weights=torchvision.models.MobileNet_V2_Weights.DEFAULT)
model.eval()

# Batch size must be 1
dummy_input = torch.randn(1, 3, 224, 224)

onnx_path = "MobilenetV2.onnx"

torch.onnx.export(
    model,                      # PyTorch model object to export
    dummy_input,                # Dummy input used for tracing (tuple is possible)
    onnx_path,                  # Output ONNX file path
    export_params=True,         # If True, saves model parameter (weight) into the ONNX file
    input_names=["input_test"], # Name of the ONNX model input tensor
    output_names=["output"],    # Name of the ONNX model output tensor
    opset_version=13            # ONNX opset version (recommended: 11 ~ 21)
)
print("✅ Save ONNX:", onnx_path)

#### 1.1.2. Verify the input name and input shape using Netron or DXTron, which allows you to visualize the model topology:

**Why Netron or DXTron is Important**

Before creating a JSON configuration file, it is critical to understand:
- The exact input tensor name
- The expected input shape
- The overall model topology

Netron/DXTron provides a convenient way to inspect ONNX models and avoids
common configuration mistakes during compilation.

You can use [netron](https://netron.app) or dxtron:
 - `Note`: You can stop the `dxtron` by clicking the stop button ('■') above!

In [ ]:
# Run DXTron
!$root_path/dx-all-suite/dx-compiler/dx_tron/DXTron-2.0.0.AppImage --no-sandbox MobilenetV2.onnx

### 1.2. JSON configuration for Input/Pre-processing/Calibration

Generate a configuration file for Input/Pre-processing/Calibration of MobilenetV2.

This JSON configuration file includes:
 - Input specifications
 - Calibration methods
 - Data preprocessing settings
 - Optional parameters for advanced compilation schemes

Model **Input Restrictions**:
 - The batch size must be fixed to 1
 - Only a single input is supported (Multi-input will be supported in 2026)
 - Input name must exactly match ONNX model definition

DX-Compiler requires strict consistency between the ONNX model and
the JSON configuration.

In the following exercises, we intentionally introduce common mistakes
to help you understand:
- Why input names must match exactly
- How data layout mismatches cause shape errors
- How to resolve these issues using preprocessing operators


#### 1.2.1. Incorrect input name case

This MobilenetV2 model's input name is `input_test` as shown below:

![](assets/mobilenetv2-input-name.png)

Input name (`input_test` in this example) must exactly match the input name of JSON configuration.

However, the following JSON configuration uses an incorrect input name (`incorrect_input_name`).

 - Correct input name: `input_test`
 - Current input name: `incorrect_input_name`

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"incorrect_input_name": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}

Let's run DX-Compiler with this wrong JSON configuration. You will encounter the following error:
> ConfigInputError: The input name in config incorrect_input_name is not same as model input input_test

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

#### 1.2.2. Incorrect input shape
In the following JSON file, the input name has the correct one - `input_test`.

However, even if your input shape is 1x3x224x224 (BxCxHxW), calibration image shape has 224x224x3 (HxWxC).

You must change the shape of calibration image to match with 1x3x224x224 (BxCxHxW) by using `transpose` and `expandDim`.

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"input_test": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}}
    ]
  }
}

Let's run DX-COM with this wrong JSON configuration. You will encounter the following error:

> ConfigInputError: Config shape [1, 3, 224, 224] does not match preprocessed data shape [1, 224, 224, 3]

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./

#### 1.2.3. Add `transpose` & `expandDim` to JSON

In [ ]:
%%writefile MobilenetV2.json
{
  "inputs": {"input_test": [1,3,224,224]},
  "calibration_num": 10,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpg","jpeg","png"],
    "preprocessings": [
      {"convertColor": {"form": "BGR2RGB"}},
      {"resize": {"width": 224, "height": 224}},
      {"div": {"x": 255.0}},
      {"normalize": {"mean": [0.485,0.456,0.406], "std": [0.229,0.224,0.225]}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

### 1.3. Compile with DX-Compiler and verify .dxnn

#### 1.3.1. Compile with `dx_com` to generate `.dxnn`

In [ ]:
!./dx_com/dx_com -m MobilenetV2.onnx -c MobilenetV2.json -o ./ &> MobilenetV2.log

#### 1.3.2. Verifying the Compiled Model

The `run_model` command performs a basic inference test to verify:
- Model correctness
- Runtime stability
- Successful deployment to the target device

In [ ]:
!run_model -m MobilenetV2.dxnn

## 2. Compiling Object Detection Model (YOLOv7) with PPU enabled
1. Reuse ONNX file from T03 custom YOLOv7 (Forklift & Worker)
2. JSON configuration for Input/Pre-processing/Calibration/PPU
3. Compile with DX-Compiler and verify .dxnn
4. Compare latency between YOLOv7 and YOLOv7 with PPU enabled

### 2.1. Reuse ONNX file from T03 custom YOLOv7 (Forklift & Worker)
- Download public YOLOv7 ONNX file
- Download the ONNX files used in the previous T03 tutorial

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

public_model_name = "YOLOV7-2.onnx"
custom_model_name = "yolov7-forklift-person.onnx"

# Check if the file exists
if not os.path.exists(public_model_name):
    # Download the ONNX file to detect both Forklifts and Workers
    !wget "sdk.deepx.ai/modelzoo/onnx/YOLOV7-2.onnx"
else:
    print(f"'{public_model_name}' already exists. Skipping download.")

# Check if the file exists
if not os.path.exists(custom_model_name):
    # Download the ONNX file to detect both Forklifts and Workers
    !wget "cs.deepx.ai/_deepx_fae_archive/dx-tutorials/yolov7-forklift-person.onnx"
else:
    print(f"'{custom_model_name}' already exists. Skipping download.")

### 2.2. JSON configuration for Input/Pre-processing/Calibration/PPU

#### 2.2.1. What is PPU?
The PPU enables hardware-accelerated post-processing for object detection models. To use the PPU,
define the ppu field in the configuration file with the appropriate type and parameters based on your
model architecture. 

#### 2.2.2. PPU parameters in JSON configuration
- **type** : `0` for anchor-based YOLO models and `1` for anchor-free YOLO models
- **conf_thres** : Confidence threshold for detection filtering (float).
> Note: This value is fixed during compilation and cannot be changed at runtime.
- **num_classes** : Number of detection classes (integer)
- **activation** : Activation function used in post-processing (commonly "Sigmoid" )
- **layer** : Dictionary mapping convolution layer node names to their anchor configurations
- Each layer entry specifies **num_anchors** : Number of anchors used in that layer
- Example:
  ```json
  {
    ... skip ...
    "ppu": {
      "type": 0,
      "conf_thres": 0.25,
      "activation": "Sigmoid",
      "num_classes": 80,
      "layer": {
        "Conv_245": {
          "num_anchors": 3
        },
        "Conv_294": {
          "num_anchors": 3
        },
        "Conv_343": {
          "num_anchors": 3
        }
      }
    }
  }
  ```

#### 2.2.3. How to set PPU configurations for YOLOv7
To configure PPU, you need to identify specific Conv operation nodes in your ONNX model:
 1. Open your model in Netron to visualize the ONNX graph
 2. Find the **three detection head Conv layers** and check each layer's name
    - These Conv layers output feature maps with shape [1, num_anchors*(5+num_classes), H, W]
    - The multiplier before (5+num_classes) is the num_anchors value you need to configure in the PPU JSON
 3. Add ppu configuration to the JSON
    - These Conv layers are typically followed by reshape, permute, and other post-processing operations

#### 2.2.4. Practice #1 find the three detection head Conv layers of Public YOLOv7

![](assets/yolov7-class-n80-ppu.png)

You can use [netron](https://netron.app) or dxtron:
 - `Note`: You can stop the `dxtron` by clicking the stop button ('■') above!

In [ ]:
# Run DXTron
!$root_path/dx-all-suite/dx-compiler/dx_tron/DXTron-2.0.0.AppImage --no-sandbox YOLOV7-2.onnx

You must find the following three Conv layer names:
  - Conv_299
  - Conv_314
  - Conv_329

#### 2.2.5. Create JSON configuration for Public YOLOv7

In [ ]:
%%writefile YOLOv7.json
{
  "inputs": {
    "images": [1, 3, 640, 640]
  },
  "calibration_method": "ema",
  "calibration_num": 100,
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"div": {"x": 255}},
      {"convertColor": {"form": "BGR2RGB"}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

In [ ]:
%%writefile YOLOv7-ppu.json
{
  "inputs": {
    "images": [1, 3, 640, 640]
  },
  "calibration_method": "ema",
  "calibration_num": 100,
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"div": {"x": 255}},
      {"convertColor": {"form": "BGR2RGB"}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  },
  "ppu": {
    "type": 0,
    "conf_thres": 0.25,
    "activation": "Sigmoid",
    "num_classes": 80,
    "layer": {
      "Conv_299": {
        "num_anchors": 3
      },
      "Conv_314": {
        "num_anchors": 3
      },
      "Conv_329": {
        "num_anchors": 3
      }
    }
  }
}

#### 2.2.6. Practice #2 - find the three detection head Conv layers of Custom YOLOv7

![](assets/yolov7-class-n2-ppu.png)

You can use [netron](https://netron.app) or dxtron:
 - `Note`: You can stop the `dxtron` by clicking the stop button ('■') above!

In [ ]:
!# Run DXTron
!$root_path/dx-all-suite/dx-compiler/dx_tron/DXTron-2.0.0.AppImage --no-sandbox yolov7-forklift-person.onnx

You must find the following three Conv layer names:
  - /model.105/m.0/Conv
  - /model.105/m.1/Conv
  - /model.105/m.2/Conv

#### 2.2.7. Create JSON configuration file

In [ ]:
%%writefile yolov7-forklift-person.json
{
  "inputs": {
    "images": [1, 3, 640, 640]
  },
  "calibration_method": "ema",
  "calibration_num": 100,
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"div": {"x": 255}},
      {"convertColor": {"form": "BGR2RGB"}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

In [ ]:
%%writefile yolov7-forklift-person-ppu.json
{
  "inputs": {
    "images": [1, 3, 640, 640]
  },
  "calibration_method": "ema",
  "calibration_num": 100,
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"div": {"x": 255}},
      {"convertColor": {"form": "BGR2RGB"}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  },
  "ppu": {
    "type": 0,
    "conf_thres": 0.25,
    "activation": "Sigmoid",
    "num_classes": 2,
    "layer": {
      "/model.105/m.0/Conv": {
        "num_anchors": 3
      },
      "/model.105/m.1/Conv": {
        "num_anchors": 3
      },
      "/model.105/m.2/Conv": {
        "num_anchors": 3
      }
    }
  }
}

### 2.3. Compile with DX-Compiler and verify .dxnn

#### 2.3.1. Compile with `dx_com` to generate `YOLOv7.dxnn` with PPU & without PPU

In [ ]:
!./dx_com/dx_com -m YOLOV7-2.onnx \
    -c YOLOv7.json -o output/public-yolov7 &> public-yolov7.log

In [ ]:
!./dx_com/dx_com -m YOLOV7-2.onnx \
    -c YOLOv7-ppu.json -o output/public-yolov7-ppu &> public-yolov7-ppu.log

#### 2.3.2. Compile with `dx_com` to generate `yolov7-forklift-person.dxnn` with PPU & without PPU

In [ ]:
!./dx_com/dx_com -m yolov7-forklift-person.onnx \
    -c yolov7-forklift-person.json \
    -o output/yolov7-forklift-person &> yolov7-forklift-person.log

In [ ]:
!./dx_com/dx_com -m yolov7-forklift-person.onnx \
    -c yolov7-forklift-person-ppu.json \
    -o output/yolov7-forklift-person-ppu &> yolov7-forklift-person-ppu.log

#### 2.3.3. Check if *.dxnn files are generated as expected

In [ ]:
!tree -h output/public-yolov7* output/yolov7-forklift-person*

2.3.4. Verify *.dxnn files using run_model

In [ ]:
!run_model -m output/public-yolov7/YOLOV7-2.dxnn -t 5

In [ ]:
!run_model -m output/public-yolov7-ppu/YOLOV7-2.dxnn -t 5

In [ ]:
!run_model -m output/yolov7-forklift-person/yolov7-forklift-person.dxnn -t 5

In [ ]:
!run_model -m output/yolov7-forklift-person-ppu/yolov7-forklift-person.dxnn -t 5

## 3. Compiling Object Detection Model (YOLOX) with PPU enabled
1. Download YOLOX ONNX files
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

#### 3.1. Download YOLOX-S ONNX file

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

# Download yoloxs model as onnx format
model_file = "YOLOX-S.onnx"
base_url = "https://sdk.deepx.ai/modelzoo/onnx/YOLOXS-1.onnx"

# If no downloaded onnx file, download from the defined URL
if not os.path.exists(model_file):
    print(f"Downloading {model_file}...")
    !wget -O $model_file $base_url
else:
    print(f"'{model_file}' already exists. Skipping download.")

### 3.2. JSON configuration for Input/Pre-processing/Calibration/PPU

#### 3.2.1. PPU `type 0` vs `tpye 1`
The PPU configuration supports the following types of object detection architectures:
  - **Type 0 (Anchor-Based YOLO)**: Designed for models that use anchor boxes, such as YOLOv3, YOLOv4, YOLOv5, and YOLOv7.
  - **Type 1 (Anchor-Free YOLO)**: Designed for anchor-free models, such as YOLOX.

#### 3.2.2. PPU parameters in JSON configuration
- **type** : `0` for anchor-based YOLO models and `1` for anchor-free YOLO models
- **conf_thres** : Confidence threshold for detection filtering (float).
> Note: This value is fixed during compilation and cannot be changed at runtime.
- **num_classes** : Number of detection classes (integer)
- **layer** : List of layer configurations, each containing
  - **bbox** : Layer name that outputs bounding box coordinates
  - **obj_conf** : Layer name that outputs object confidence scores
  - **cls_conf** : Layer name that outputs class-wise confidence scores
- Example:
  ```json
  {
    ... skip ...
    "ppu": {
      "type": 1,
      "conf_thres": 0.25,
      "num_classes": 80,
      "layer": [
        {
          "bbox": "Conv_261",
          "obj_conf": "Conv_262",
          "cls_conf": "Conv_254"
        },
        {
          "bbox": "Conv_282",
          "obj_conf": "Conv_283",
          "cls_conf": "Conv_275"
        },
        {
          "bbox": "Conv_303",
          "obj_conf": "Conv_304",
          "cls_conf": "Conv_296"
        }
      ]
    }
  }
  ```

#### 3.2.3. How to set PPU configurations for YOLOX (Anchor-free model)

To configure PPU, you need to identify specific Conv operation nodes in your ONNX model:
 1. Open your model in Netron to visualize the ONNX graph
 2. Find the three sets of `bbox`, `obj_conf`, and `cls_conf` values and check each layer's name
    - bbox : Conv layer that outputs bounding box regression values
    - obj_conf : Conv layer that outputs objectness confidence scores
    - cls_conf : Conv layer that outputs class prediction scores 
 3. Add ppu configuration to the JSON
    - These three branches run in parallel for each scale level
    - This results in a total of 9 Conv layers (3 scales × 3 branches per scale)

#### 3.2.4. Practice - find the three sets of `bbox`, `obj_conf`, and `cls_conf` values

![](assets/yolox-class-n80-ppu.png)

You can use [netron](https://netron.app) or dxtron:
 - `Note`: You can stop the `dxtron` by clicking the stop button ('■') above!

In [ ]:
!# Run DXTron
!$root_path/dx-all-suite/dx-compiler/dx_tron/DXTron-2.0.0.AppImage --no-sandbox YOLOX-S.onnx

You must find the following nine Conv layer names:

<table align="left" border="1" style="width: 550px;">
  <tr style="background-color: #D0D0D0;"><th>Scale lvl</th><th>bbox</th><th>obj_conf</th><th>cls_conf</th></tr>
  <tr align="center"><td>80x80</td><td>Conv_261</td><td>Conv_262</td><td>Conv_254</td></tr>
  <tr align="center"><td>40x40</td><td>Conv_282</td><td>Conv_283</td><td>Conv_275</td></tr>
  <tr align="center"><td>20x20</td><td>Conv_303</td><td>Conv_304</td><td>Conv_296</td></tr>
</table>

#### 3.2.5. Create JSON configuration for Public YOLOX-S

In [ ]:
%%writefile YOLOX-S.json
{
  "inputs": {
    "images": [1, 3, 640, 640]
  },
  "calibration_method": "ema",
  "calibration_num": 100,
  "num_samples": 1024,
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

In [ ]:
%%writefile YOLOX-S-PPU.json
{
  "inputs": {
    "images": [1, 3, 640, 640]
  },
  "calibration_method": "ema",
  "calibration_num": 100,
  "num_samples": 1024,
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  },
  "ppu": {
    "type": 1,
    "conf_thres": 0.25,
    "num_classes": 80,
    "layer": [
      {
        "bbox": "Conv_261",
        "obj_conf": "Conv_262",
        "cls_conf": "Conv_254"
      },
      {
        "bbox": "Conv_282",
        "obj_conf": "Conv_283",
        "cls_conf": "Conv_275"
      },
      {
        "bbox": "Conv_303",
        "obj_conf": "Conv_304",
        "cls_conf": "Conv_296"
      }
    ]
  }
}

### 3.3. Compile with DX-Compiler and verify .dxnn

#### 3.3.1. Compile both YOLOX-S non-PPU and YOLOX-S PPU

In [ ]:
!./dx_com/dx_com -m YOLOX-S.onnx -c YOLOX-S.json -o output/yolox-s &> yolox-s.log

In [ ]:
!./dx_com/dx_com -m YOLOX-S.onnx -c YOLOX-S-PPU.json -o output/yolox-s-ppu &> yolox-s-ppu.log

3.3.2. Check if both **yolox-s/YOLOX-S.dxnn** and **yolox-s-ppu/YOLOX-S.dxnn** are generated as expected:

In [ ]:
!tree output/yolox*

#### 3.3.3. Verify them using run_model

In [ ]:
!run_model -m output/yolox-s/YOLOX-S.dxnn -t 5

In [ ]:
!run_model -m output/yolox-s-ppu/YOLOX-S.dxnn -t 5

## 4. Compiling Object Detection Model (YOLOv9-S)
1. Export PyTorch → ONNX
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

### 4.1. Export PyTorch → ONNX

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

#### 4.1.1. Download `yolov9-s.pt` PyTorch model to your local system:

In [ ]:
# torch version down to fix yolov9 version conflict issue
!pip install torch==2.5.1 torchvision==0.20.1
!pip install onnx onnxsim

In [ ]:
!wget --no-check-certificate https://github.com/WongKinYiu/yolov9/releases/download/v0.1/yolov9-s.pt

#### 4.1.2. Export the downloaded torch model to ONNX:

Download yolov9 git repo to use `export.py` for yolov9.

In [ ]:
!git clone https://github.com/WongKinYiu/yolov9.git

Export PyTorch based `yolov9-s` model to ONNX

In [ ]:
!cd yolov9 && python3 export.py --weights ../yolov9-s.pt \
                                --img-size 640 640 \
                                --opset 12 \
                                --batch-size 1 \
                                --include onnx

Check if `yolov9-s.onnx` file is generated.

In [ ]:
!ls | grep yolov9-s.onnx

### 4.2. JSON c.onfiguration for Input/Pre-processing/Calibration

You can use [netron](https://netron.app) or dxtron:
 - `Note`: You can stop the `dxtron` by clicking the stop button ('■') above!

In [ ]:
!# Run DXTron
!$root_path/dx-all-suite/dx-compiler/dx_tron/DXTron-2.0.0.AppImage --no-sandbox yolov9-s.onnx

In [ ]:
%%writefile yolov9-s.json
{
  "inputs": {"images": [1,3,640,640]},
  "calibration_num": 100,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"div": {"x": 255}},
      {"convertColor": {"form": "BGR2RGB"}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

### 4.3. Compile with DX-Compiler and verify .dxnn

In [ ]:
!./dx_com/dx_com -m yolov9-s.onnx -c yolov9-s.json -o output &> yolov9-s.log

Check if **yolov9-s.dxnn** file is generated as expected:

In [ ]:
!ls | grep output/yolov9-s.dxnn

In [ ]:
!run_model -m output/yolov9-s.dxnn -t 10

## 5. Compiling Object Detection Model (YOLOv11)
1. Download YOLOv11 ONNX files
2. JSON configuration for Input/Pre-processing/Calibration
3. Compile with DX-Compiler and verify .dxnn

#### 5.1. Download YOLOv11 ONNX files

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

# Download yolov11l/m/s models as onnx format
model_files = ["YOLOV11L.onnx", "YOLOV11M.onnx", "YOLOV11S.onnx"]
base_url = "cs.deepx.ai/_deepx_fae_archive/dx-tutorials"

# If no downloaded onnx files, download from the defined URL
for file_name in model_files:
    if not os.path.exists(file_name):
        full_url = f"{base_url}/{file_name}"
        print(f"Downloading {file_name}...")
        !wget $full_url
    else:
        print(f"'{file_name}' already exists. Skipping download.")

#### 5.2. JSON configuration for Input/Pre-processing/Calibration

You can use [netron](https://netron.app) or dxtron:
 - `Note`: You can stop the `dxtron` by clicking the stop button ('■') above!

In [ ]:
!!# Run DXTron
!$root_path/dx-all-suite/dx-compiler/dx_tron/DXTron-2.0.0.AppImage --no-sandbox YOLOV11S.onnx

In [ ]:
%%writefile YOLOV11S.json
{
  "inputs": {"images": [1,3,640,640]},
  "calibration_num": 100,
  "calibration_method": "ema",
  "default_loader": {
    "dataset_path": "./calibration_dataset",
    "file_extensions": ["jpeg","jpg","png","JPEG"],
    "preprocessings": [
      {"resize": {"mode": "pad", "size": 640, "pad_location": "edge", "pad_value": [114,114,114]}},
      {"div": {"x": 255}},
      {"convertColor": {"form": "BGR2RGB"}},
      {"transpose": {"axis": [2,0,1]}},
      {"expandDim": {"axis": 0}}
    ]
  }
}

#### 5.3. Compile with DX-Compiler and Verify .dxnn

In [ ]:
!./dx_com/dx_com -m YOLOV11S.onnx -c YOLOV11S.json -o output &> YOLOV11S.log

In [ ]:
!ls | grep output/YOLOV11S.dxnn

In [ ]:
!run_model -m output/YOLOV11S.dxnn -t 10

## 6. Compiling ViT(Vision Transformer) Model

https://github.com/mlfoundations/open_clip


In [ ]:
!pip install -q open_clip_torch
!pip install -q onnxruntime onnxsim onnx onnxscript

In [ ]:
# Move to "dx-tutorials/dx-all-suite/dx-compiler/dx_com"
import os
root_path = os.environ.get('ROOT_PATH')
%cd $root_path/dx-all-suite/dx-compiler/dx_com

#### 6.1. Download ONNX file from Hugging Face
You will download `ViT-L-14-quickgelu-dfn2b` model and JSON configuration will be generated.

In [ ]:
# 1) Download the ONNX file from Hugging Face *without hardcoding any HF URL*
# 2) Save it as "ViT-L-14-quickgelu-dfn2b.onnx"
# 3) Generate a JSON config using the existing get_config() routine

import json
import shutil
from pathlib import Path

import torch
import torchvision.transforms as T
from huggingface_hub import hf_hub_download


def parse_preprocess(preprocess):
    dx_preprocess = []

    for trf in preprocess.transforms:
        trf_name = trf.__class__.__name__

        if trf_name == "Resize":
            size = trf.size
            if isinstance(size, int):
                dx_preprocess.append({"resize": {"width": size, "height": size}})
            else:
                dx_preprocess.append({"resize": {"width": size[0], "height": size[1]}})

        elif trf_name == "ToTensor":
            dx_preprocess.append({"div": {"x": 255.0}})

        elif trf_name == "Normalize":
            dx_preprocess.append({"normalize": {"mean": trf.mean, "std": trf.std}})

        # torchvision uses Lambda instead of a raw "function" transform
        elif trf_name == "Lambda":
            # We only support RGB conversion lambda in this demo
            dx_preprocess.append({"convertColor": {"form": "BGR2RGB"}})

        elif trf_name == "CenterCrop":
            size = trf.size
            if isinstance(size, int):
                dx_preprocess.append({"centercrop": {"width": size, "height": size}})
            else:
                dx_preprocess.append({"centercrop": {"width": size[0], "height": size[1]}})

        else:
            raise NotImplementedError(f"Unsupported transform: {trf_name}")

    return dx_preprocess


def get_config(img: torch.Tensor, preprocess):
    template = {
        "inputs": {"image": [1, 3, 224, 224]},
        "default_loader": {
            "dataset_path": "calibration_dataset/",
            "file_extensions": ["jpeg", "jpg", "png", "JPEG"],
        },
        "calibration_num": 100,
        "calibration_method": "ema",
        "train_batchsize": 32,
        "num_samples": 100,
    }

    _preprocess = parse_preprocess(preprocess)
    _preprocess.append({"transpose": {"axis": [2, 0, 1]}})
    _preprocess.append({"expandDim": {"axis": 0}})

    template["default_loader"]["preprocessings"] = _preprocess
    return template


# --- Settings ---
OUTPUT_DIR = Path("./")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Verified ViT Models - You can change ViT models depending on your use cases
# Model              | Pre-trained
# ----------------------------------------
# ViT-B-32-quickgelu | metaclip_fullcc
# ViT-B-16-quickgelu | metaclip_fullcc
# ViT-L-14-quickgelu | dfn2b (*selected)
# ViT-B-16           | dfn2b
# ViT-L-14           | datacomp_xl_s13b_b90k
# ViT-B-32-256       | datacomp_s34b_b86k
# ViT-L-14-336       | openai
MODEL_NAME = "ViT-L-14-quickgelu-dfn2b.onnx"
ONNX_DST = OUTPUT_DIR / MODEL_NAME
JSON_DST = OUTPUT_DIR / (Path(MODEL_NAME).stem + ".json")

# This repo hosts the ONNX under "visual/model.onnx"
REPO_ID = "immich-app/ViT-L-14-quickgelu__dfn2b"
REPO_FILENAME = "visual/model.onnx"

# --- 1) Download ONNX (cached by huggingface_hub) ---
cached_path = Path(
    hf_hub_download(
        repo_id=REPO_ID,
        filename=REPO_FILENAME,
        # Set to True if you want to avoid any network calls (must be already cached).
        # local_files_only=True,
    )
)

# Copy/rename to the filename we want to show in the tutorial
if not ONNX_DST.exists() or ONNX_DST.stat().st_size != cached_path.stat().st_size:
    shutil.copyfile(cached_path, ONNX_DST)

print(f"ONNX ready: {ONNX_DST} ({ONNX_DST.stat().st_size:,} bytes)")


# --- 2) Build a CLIP-like preprocessing pipeline (educational default) ---
# Note: CLIP commonly uses 224x224 and the following mean/std.
clip_mean = (0.48145466, 0.4578275, 0.40821073)
clip_std  = (0.26862954, 0.26130258, 0.27577711)

preprocess = T.Compose([
    T.Resize(224),
    T.CenterCrop(224),
    T.Lambda(lambda img: img.convert("RGB")),  # recorded as "BGR2RGB" in our parse
    T.ToTensor(),
    T.Normalize(mean=clip_mean, std=clip_std),
])

dummy_img = torch.empty(1, 3, 224, 224)

config = get_config(dummy_img, preprocess)
with open(JSON_DST, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=2)

print(f"Config saved: {JSON_DST}")


#### 6.2. Double-check JSON Configuration & Model Topology using DXTron

In [ ]:
!cat ViT-L-14-quickgelu-dfn2b.json

In [ ]:
!# Run DXTron
!$root_path/dx-all-suite/dx-compiler/dx_tron/DXTron-2.0.0.AppImage \
    --no-sandbox ViT-L-14-quickgelu-dfn2b.onnx

#### 6.3. Compile

In [ ]:
!./dx_com/dx_com -m ViT-L-14-quickgelu-dfn2b.onnx \
    -c ViT-L-14-quickgelu-dfn2b.json -o output &> ViT-L-14-quickgelu-dfn2b.log

#### 6.4. Verifying the Compiled Model

In [ ]:
!run_model -m output/ViT-L-14-quickgelu-dfn2b.dxnn -l 100

## 7. Summary

In this tutorial, you learned how to:
- Convert PyTorch models to ONNX
- Configure preprocessing and calibration using JSON
- Enable PPU for object detection models
- Compile various model architectures using DX-Compiler

You are now ready to integrate DX-Compiler into your production workflow.